# Препроцессинг данных

Загружаем библиотеки

In [ ]:
%matplotlib inline
!pip install pymorphy2
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import json
from tqdm import tqdm
from sklearn.metrics import *
import warnings
warnings.filterwarnings("ignore")

# Sentiment Analysis - Анализ тональности

Сегодня мы познакомимся с основами NLP на примере задачи анализа тональность (Sentiment Analysis) с соревнования [Kaggle Competition](https://www.kaggle.com/c/sentiment-analysis-in-russian/data) .

Задача соревнования - построить ML модель, способную различать тональность (позитивная, негативная, нейтральная) новостей.

## Загружаем данные

Данные записаны в формате `json`. Для его чтения воспользуемся библиотекой `json` и методом `open`.

In [ ]:
!unzip kazakh_news.zip

In [ ]:
# Загружаем данные
with open('train.json', encoding = 'utf-8') as json_file:
    data = json.load(json_file)

In [ ]:
len(data)

In [ ]:
data[0]

Будем решать задачу классификации на 3 класса.

In [ ]:
print(set([x["sentiment"] for x in data]))

Каждый пример для обучения состоит из id, текстового фрагмента новости, и лейбла (`'positive', 'negative', 'neutral'`).

Задача - предсказать лейбл.

In [ ]:
# Посмотрим на пример
num = 1

print("ID: ",          data[num]["id"], "\n")
print("Text: \n",      data[num]["text"])
print("Sentiment: ",   data[num]["sentiment"], "\n")

Посмотрим на соотношение классов. Видим, что в данных присутсвует сильный дисбаланс. Почти половина всех новостей - нейтральные, а негативных новостей меньше всего.

In [ ]:
from collections import Counter
print(len(data))
Counter([x['sentiment'] for x in data])

# Предобработка данных
Прежде, чем перейти к ML, текст необходимо предобработать.

## Шаг 1. Токенизация и удаление стоп-слов
Первый шаг предобработки - разбить текст на единицы, с которыми мы будем работать. Эти единицв называются **токенами (tokens)**, а процесс - **токенизация (tokenization)**. В большинстве случаев в качестве токенов используют слова, но иногда работают с буквами.

Будем работать со словами. Проще всего разбить текст на слова по пробелам (не забывая про пунктуацию).


## Шаг 2. Удаляем стоп-слова


**Стоп-слова** – это слова, которые выкидываются из текста при обработке текста. Когда мы применяем машинное обучение к текстам, такие слова могут добавить много шума, поэтому необходимо избавляться от нерелевантных слов.

Стоп-слова это обычно понимают артикли, междометия, союзы и т.д., которые не несут смысловой нагрузки. При этом надо понимать, что не существует универсального списка стоп-слов, все зависит от конкретного случая.





In [ ]:
import nltk   # Natural Language Toolkit

In [ ]:
# загружаем список стоп-слов для русского
nltk.download('stopwords')
stop_words = nltk.corpus.stopwords.words('russian')

# примеры стоп-слов
print(len(stop_words))
print(stop_words[:10])

Инициализируем `WordPunctTokenizer`, с помощью которого затем разобьем текст на слова.

In [ ]:
word_tokenizer = nltk.WordPunctTokenizer()
tokens = word_tokenizer.tokenize('казнить нельзя, помиловать!?!!')
print(tokens)


Запишем предобработку текста в виде функции.

In [ ]:
import re
regex = re.compile(r'[А-Яа-яA-zёЁ-]+')

def words_only(text, regex=regex):
    try:
        return " ".join(regex.findall(text)).lower()
    except:
        return ""

words_only('Казнить, нельзя помиловать!!! 2023 год')

In [ ]:
# расширим список стоп-слов, словами, которые являеются стоп-словами в данной задаче
add_stop_words = ['kz', 'казахстан', 'астана', 'казахский', 'алматы', 'ао', 'оао', 'ооо']
months = ['январь', 'февраль', 'март', 'апрель', 'май', 'июнь', 'июль', 'август', 'сентябрь', 'октябрь', 'ноябрь', 'декабрь',]
all_stop_words = stop_words + add_stop_words + months

def process_data(data):
    texts = []
    targets = []

    # поочередно проходим по всем новостям в списке
    for item in tqdm(data):

        text_lower = words_only(item['text']) # оставим только слова
        tokens     = word_tokenizer.tokenize(text_lower) #разбиваем текст на слова

        # удаляем пунктуацию и стоп-слова
        tokens = [word for word in tokens if (word not in all_stop_words and not word.isnumeric())]

        texts.append(tokens) # добавляем в предобработанный список

    return texts

In [ ]:
# запускаем нашу предобработку
y = [item['sentiment'] for item in data]
texts = process_data(data)

Теперь каждый пример представлен списком слов. Причем все слова с маленькой буквы. Пунктуацию и стоп-слова мы удалили.

In [ ]:
# example
i = 2
print("Label: ", y[i])
print("Tokens: ", texts[i][:5])

## Шаг 3 . Нормализация слов

Обычно тексты содержат разные грамматические формы одного и того же слова, а также могут встречаться однокоренные слова. Чтобы унифицировать слова в тексте и избавиться от различных форм слова, слова в тексте можно нормализовать.


Существует 2 наиболее известных способа нормализации слов: **стемминг (stemming)** и **лемматизация (лемматизация)**.

В общих чертах они похоже, но между этими методами есть различия. В зависимости от языка и задачи тот или иной метод может быть предпочтительнее.

1) **Стемминг** (англ. stemming — находить происхождение) — это процесс нахождения основы слова для заданного исходного слова. Основа слова не обязательно совпадает с морфологическим корнем слова и не обязана являться существующим словом в языке. Стемминг – это грубый эвристический процесс, который отрезает «лишнее» от корня слов, часто это приводит к потере словообразовательных суффиксов.

2) **Лемматизация** приводит все встречающиеся словоформы к одной, нормальной словарной форме. **Лемматизация** использует словарь и морфологический анализ, чтобы в итоге привести слово к его канонической форме – **лемме**.


In [ ]:
from nltk.stem.snowball import SnowballStemmer

# инициализируем стеммер
stemmer = SnowballStemmer("russian")

In [ ]:
# примеры стемминга
i = 1
for aword in texts[i][:10]:
    aword_stem = stemmer.stem(aword)
    print("ДО: %s, ПОСЛЕ: %s" % (aword, aword_stem))

In [ ]:
text = 'в этот прекрасный солнечный день мы сидим на семинаре по обработке естественного языка в университете имени Витте'
stemmed_text = ' '.join([stemmer.stem(x) for x in text.split(' ')])
print('Исходная строка:\t',text)
print('Обрубленная строка:\t',stemmed_text)

In [ ]:
stemmer = SnowballStemmer("english")
text = 'On this beautiful sunny day we are sitting in a natural language processing seminar at Witte University'
stemmed_text = ' '.join([SnowballStemmer("english").stem(x) for x in text.split(' ')])
print('Original text:\t',text)
print('Stemmed text:\t',stemmed_text)

Видно, что для русского языка результат не очень хороший. Попробуем что-нибудь получше.


In [ ]:
# загружаем библиотеку для лемматизации
import pymorphy2 # Морфологический анализатор

# инициализируем лемматизатор :)
morph = pymorphy2.MorphAnalyzer()

Посмотрим на примерах, как работает лемматизация.

In [ ]:
 morph.parse('студентами')

[здесь](https://pymorphy2.readthedocs.io/en/stable/user/grammemes.html#grammeme-docs) можно посмотреть список граммем


In [ ]:
 morph.parse('лук')

In [ ]:
i = 1
for aword in texts[i][:10]:
    aword_norm = morph.parse(aword)[0].normal_form
    print("Исходное слово: %s\tЛемматизированное: %s" % (aword, aword_norm))

In [ ]:
text = 'в этот прекрасный солнечный день мы сидим на семинаре по обработке естественного языка в университете имени Витте'
stemmed_text = ' '.join([morph.parse(x)[0].normal_form for x in text.split(' ')])
print('Оригинальный текст:\t',text)
print('Лемматизированный текст:\t',stemmed_text)

Теперь давайте лемматизируем все тексты.

## Be careful! Осторожно
**Библиотека Pymorphy может работать. Чтобы не ждать вы можете загрухить уже лемматизированный текст**

In [ ]:
from tqdm import tqdm_notebook

# применяем лемматизацию ко всем текстам
for i in tqdm_notebook(range(len(texts))):           # tqdm_notebook создает шкалу прогресса :)
    text_lemmatized = [morph.parse(x)[0].normal_form for x in texts[i]] # применяем лемматизацию для каждого слова в тексте
    texts[i] = ' '.join(text_lemmatized)                # объединяем все слова в одну строку через пробел

Чтобы не ждать загружаем предобработанные тексты :)

In [ ]:
texts = [x.replace('\n','') for x in open('text_lemmatized.txt', encoding = 'utf-8').readlines()]

Посмотрим на пример.

In [ ]:
# посмотрим на пример
i = 123
print("Label: ",   y[i])
print("Text: \n",  texts[i])

# Моделирование & Векторные представления

## Разбиваем на train&test

Лейблы у нас также закодированы словами. Для корректной работы алгорима конвертируем их в числа (`'negative', 'neutral', 'positive'`):

    negative = -1
    neutral  = 0
    positive = 1

In [ ]:
# Функция для кодирования лейблов
def label2num(y):
    if y == 'positive':
        return 1
    if y == 'negative':
        return -1
    if y == 'neutral':
        return 0

encoded_y = [label2num(yy) for yy in y]

**отложим часть данных для тестирования и оценки качества алгоритма. Для этого воспользуемся функцией `train_test_split`.

In [ ]:
#train test_split
from sklearn.model_selection import train_test_split
train_texts, test_texts, train_y, test_y = train_test_split(texts, encoded_y, test_size=0.2, random_state=42, stratify = y)

# Bag of Words

**Bag of Words или мешок слов** — это модель, представляющая собой неупорядоченный набор слов, входящих в обрабатываемый текст.


Часто модель представляют в виде матрицы, в которой строки соответствуют отдельному тексту, а столбцы — входящие в него слова. Ячейки на пересечении являются числом вхождения данного слова в соответствующий документ. Данная модель удобна тем, что переводит человеческий язык слов в понятный для компьтера язык цифр.


Модель Bag of Words реализована в библиотеке `sklearn` в классе `feature_extraction.text.CountVectorizer`.

In [ ]:
#Инициализируем векторайзер
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer(ngram_range=(1,3), max_features = 200)
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

In [ ]:
vectorizer.vocabulary_

In [ ]:
len(vectorizer.vocabulary_)

In [ ]:
print(vectorizer.get_feature_names_out())
len(vectorizer.get_feature_names_out())


Обучаем `vectorizer` на train-данных и сразу преобразем их в вектора с помощью метода `fit_transform`.

In [ ]:
# Обучаем vectorizer на train-данных и сразу преобразем их в вектора с помощью метода fit_transform
train_X = vectorizer.transform(train_texts)
train_X.todense()[:2]

In [ ]:

type(train_X)

Также применяем обученный `vectorizer` к данным для тестирования.

In [ ]:
test_X  = vectorizer.transform(test_texts)

Теперь каждое предложение задано в виде вектора! Можем строить классификатор!

# Обучаем классификатор.

В качестве классификатора будем использовать **Random Forest**.


In [ ]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)

In [ ]:
print('Предсказанные метки: ', pred[0:20], ".....")
print('Истинные метки: ', test_y[0:20], ".....")

Ничего не понятно! Приведем метки в нормальный вид!

In [ ]:
# Функция для кодирования лейблов
def num2label(y):
    if y == 1:
        return 'positive'
    if y == -1:
        return 'negative'
    if y == 0:
        return 'neutral'

decoded_pred = [num2label(y) for y in pred]
decoded_test_y = [num2label(y) for y in test_y]
print('Предсказанные метки: ', decoded_pred[0:20], ".....")
print('Истинные метки: ', decoded_test_y [0:20], ".....")

### Оценка качества

Качество классификатора будем оценивать по метрикам accuracy и f1.



In [ ]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('Precision: ', precision_score(test_y, pred, average='weighted'))
print('Recall: ', recall_score(test_y, pred, average='weighted'))
print('F1: ', f1_score(test_y, pred, average = 'weighted'))

# Посмотрим на несколько примеров

In [ ]:
for i in range(10):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст новости: ', train_texts[i][:500]+'...')
    print('\n')

# Bonus: TF-IDF - вектора чуть поумнее

**TF-IDF** (от англ. TF — term frequency, IDF — inverse document frequency) — статистическая мера, используемая для оценки важности слова в контексте документа, являющегося частью коллекции документов или корпуса. Вес некоторого слова пропорционален частоте употребления этого слова в документе и обратно пропорционален частоте употребления слова во всех документах коллекции.


**Term Frequency** число раз терм $t$ встречается в документе $d$.
$$
TF_{t,d} = term\!\!-\!\!frequency(t, d)
$$

**Inverse Document Frequency** мера того, сколько информации несет данное слово. Иными словами, частотные слова, содержащиеся во всех документах несут мало информации, в то время как слова частотные лишь в ограниченном числе документов содержат большое количество информации об этих документах. **IDF** - это инверсия частоты, с которой некоторое слово встречается в документах коллекции.


$$
IDF_t = inverse\!\!-\!\!document\!\!-\!\!frequency(t) = \log \frac{N}{DF_t}
$$

$N$ - число документов в корпусе.

$DF_t$ - число документов содержащих слово $t$.



$$
TF\!\!-\!\!IDF_{t,d} = TF_{t,d} \times IDF_t
$$

TF-IDF оценивает важность слов в корпусе документов.

Модель TF-IDF реализована в библиотеке `sklearn` в классе `feature_extraction.TfidfVectorizer`.

In [ ]:
#вычисляем tf-idf
from sklearn.feature_extraction.text import TfidfVectorizer
# Fit TF-IDF on train texts
vectorizer = TfidfVectorizer(max_features = 200, norm = None) # возмем топ 200 слов
vectorizer.fit(train_texts)

# Топ-10 слов
vectorizer.get_feature_names_out()[:10]

In [ ]:
# Обучаем TF-IDF на train, а затем применяем к train и test
train_X = vectorizer.fit_transform(train_texts)
test_X  = vectorizer.transform(test_texts)

In [ ]:
# Пример
train_X.todense()[:2] # посмотрим на первые 2 строки

## Обучаем классификатор

In [ ]:
#import алгоритма из библиотеки
from sklearn.ensemble import RandomForestClassifier

# инициализируем модель
clf = RandomForestClassifier(n_estimators = 500, max_depth = 10)

# обучаем ее на тренировочных данных
clf = clf.fit(train_X, train_y)

# делаем предсказание для тестовых данных
pred = clf.predict(test_X)

In [ ]:
print('Предсказанные метки: ', pred[0:20], ".....")
print('Истинные метки: ', test_y[0:20], ".....")

In [ ]:
# Функция для кодирования лейблов
def num2label(y):
    if y == 1:
        return 'positive'
    if y == -1:
        return 'negative'
    if y == 0:
        return 'neutral'

decoded_pred = [num2label(y) for y in pred]
decoded_test_y = [num2label(y) for y in test_y]
print('Предсказанные метки: ', decoded_pred[0:20], ".....")
print('Истинные метки: ', decoded_test_y [0:20], ".....")

In [ ]:
print('Accuracy: ', accuracy_score(test_y, pred))
print('F1: ', f1_score(test_y, pred, average = 'macro'))

## Посмотрим на несколько примеров

In [ ]:
for i in range(10):
    print('Истинный лейбл:',decoded_test_y[i])
    print('Предсказанный лейбл:',decoded_pred[i])
    print('Текст новости: ', train_texts[i][:500]+'...')
    print('\n')